In [1]:
import boto3
import pandas as pd
import io
import numpy as np
from datetime import datetime, timedelta

In [2]:
siteid_dict = {
    '555 Taxter':'US-NY-010523-1901',
    '565 Taxter':'US-NY-010523-1900',
    '4WRO':'US-NY-010604-2000',
    '660 WP':'US-NY-010591-1900'
} 

In [3]:
#lists of past events and holidays that need to be referred to when finding CBl days

past_activations = ['2022-06-29', '2023-02-28', '2023-05-25', '2023-06-29', '2023-07-27', 
                    '2023-09-06', '2023-09-07', '2024-01-17','2024-04-17'] 

daybefore_activations = [(datetime.strptime(date, '%Y-%m-%d') - timedelta(days=1)).strftime('%Y-%m-%d') for date in past_activations]

NYISO_holidays = ['2024-01-01', '2024-01-15', '2024-02-19', '2024-05-27' ]

past_activations = pd.to_datetime(past_activations, errors='coerce')
daybefore_activations = pd.to_datetime(daybefore_activations, errors='coerce')
NYISO_holidays = pd.to_datetime(NYISO_holidays, errors='coerce')

In [4]:
def get_30days(site_id, event_day):
    session = boto3.Session()
    s3_client = session.client('s3')
    bucket_name = 'rtac-ddr'

    sub_folder_prefix = f'processed/{site_id}/1s/daily/'

    try:
        # Convert event_day to datetime object
        event_day_date = datetime.strptime(event_day, '%Y-%m-%d')
        start_date = event_day_date - timedelta(days=30)

        # List objects in the specified sub-folder
        response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=sub_folder_prefix)

        data_frames = []

        if 'Contents' in response:
            for obj in response['Contents']:
                # Extract the file name from the full key path
                file_name = obj['Key'].split('/')[-1]

                # Extract the date from the filename (assuming format: US-NY-010523-1900-YYYYMMDD-YYYYMMDD-Aggregated.csv)
                parts = file_name.split('-')
                if len(parts) >= 5:
                    file_date_str = parts[4]
                    try:
                        file_date = datetime.strptime(file_date_str, '%Y%m%d')
                        if start_date <= file_date <= event_day_date:
                            file_obj = s3_client.get_object(Bucket=bucket_name, Key=obj['Key'])
                            file_content = file_obj['Body'].read()
                            data_frame = pd.read_csv(io.BytesIO(file_content))
                            data_frames.append(data_frame)
                    except ValueError:
                        print(f"Skipping file with invalid date format: {obj['Key']}")
        else:
            print(f"The sub-folder {sub_folder_prefix} in bucket {bucket_name} is empty.")

        # Aggregate all data frames into a single data frame
        if data_frames:
            combined_df = pd.concat(data_frames, ignore_index=True)
            combined_df['generateTime'] = pd.to_datetime(combined_df['generateTime'])
            combined_df.set_index('generateTime', inplace=True)

            hourly_aggregations = {
    
                'OPR_DT': 'first',
                'OPR_HR':'first',
                'Net Load': 'mean',
                'ESS Output': 'mean',
                'Gross Load': 'mean',
                'SoC': 'last',
                'SoH': 'last'
            }
            thirtyday_hourly = combined_df.resample('h').agg(hourly_aggregations)
            return thirtyday_hourly
        else:
            print("No files found in the specified date range.")
            return pd.DataFrame()
    except s3_client.exceptions.NoSuchBucket:
        print(f"The bucket {bucket_name} does not exist.")
    except Exception as e:
        print(f"An error occurred: {e}")
        return pd.DataFrame()
    
# Example usage:
# df = get_30days('site123', '2023-05-25')
    
patu = get_30days('US-NY-010604-2000', '2024-05-25')

In [97]:
def cbl_basis_calculator(thirtydays_df, event_day, event_start_hour, event_end_hour):
    event_day = pd.to_datetime(event_day)
    # Create an empty dataframe in which to store the CBL values 
    ten_eligible_days = []
    
    # get initial seed value for average event period usage level, the one hour with the highest demand (highest net load)
    initial_seed_value = thirtydays_df.loc[(thirtydays_df['OPR_HR'] >= event_start_hour) & (thirtydays_df['OPR_HR'] <= event_end_hour), 'Net Load'].max()
    event_period_usage_level = initial_seed_value
    # Begin with weekday that is 2 days prior to the event. If it is a holiday, an event day, a day before an event day, or a weekend, remove it from the dataset
    reversed_thirtydays = thirtydays_df.iloc[::-1]
    reversed_thirtydays['OPR_DT'] = pd.to_datetime(reversed_thirtydays['OPR_DT'], errors='coerce')
    reversed_thirtydays['is_weekend'] = reversed_thirtydays['OPR_DT'].apply(lambda x: x.weekday() > 4)
    
         # don't include any past events
         # skip the day before the current event
         # skip days right before an event
         # skip holidays
         # skip weekends
    mask = reversed_thirtydays['OPR_DT'].isin(past_activations) | \
           (reversed_thirtydays['OPR_DT'] == (event_day - timedelta(days=1))) | \
           reversed_thirtydays['OPR_DT'].isin(daybefore_activations)| \
           reversed_thirtydays['OPR_DT'].isin(NYISO_holidays)| \
           reversed_thirtydays['is_weekend']
    
    #only look at the event hours for CBL numbers
    mask2 = ~reversed_thirtydays['OPR_HR'].between(event_start_hour, event_end_hour)
    
    #print(f"Mask:\n{mask.head(48)}")
    #print(reversed_thirtydays[~mask])

    filtered_df = reversed_thirtydays[~mask]
    filtered_df = filtered_df.loc[~mask2]
    groupedby_day = filtered_df.groupby('OPR_DT').mean().sort_index(ascending=False)
    
    daily_usage_levels = []
    total_sum = sum(daily_usage_levels)
    count = 0

    #Iterate over the filtered DataFrame
    for row in groupedby_day.iterrows():
        # Calculate net load average for event period 
        dailyevent_avg = row[1]['Net Load'].sum()
        #eliminate low usage days (less than 25% of seed value or event period usage level)
        if dailyevent_avg < (event_period_usage_level*0.25):
            continue
        # replace the initial seed value if it is the first day
        if event_period_usage_level == initial_seed_value:
            event_period_usage_level = dailyevent_avg
            daily_usage_levels.append(event_period_usage_level)
            count += 1
        # for the following days, add them to the average calculation for average event period usage level
        else:
            daily_usage_levels.append(dailyevent_avg)
            count += 1
        # also add them to the CBL window dataframe
        ten_eligible_days.append(row[1])
        
        if count == 10:
            break
       #elif count < 10:
            #eliminate seed value and low usage step
    pf = pd.DataFrame(ten_eligible_days)
    top_5 = pf.nlargest(5, 'Net Load')
    filtered_top5 = filtered_df.loc[filtered_df.index.normalize().isin(top_5.index)]
    CBL_basis = filtered_top5.groupby(['OPR_HR']).agg({'Net Load': 'mean'})
    return CBL_basis
    # Take the 5 days with the highest average daily event period usage, then for each hour of the event, take the average usage in those top 5 days. This is the CBL basis.


In [98]:
cbl_basis_calculator(patu,'2024-05-25',13,17)

,Net Load
OPR_HR,
13,348.981097
14,340.157619
15,344.943614
16,345.401002
17,331.878649


In [ ]:
# one for weekday, one for weekend

# Inputs:
# site, event day, event start time (hour ending), event end time (hour ending)

# Dataframes:
# DF 1: 30 days data from event and before - hourly
# DF 2: CBL basis table - 5 days with highest average daily event period usages, for the hours of the event
# DF 3: CBL values: average of each hour of DF 2
# DF 4: adjustment hours values
# DF 5: adjusted CBLs table

# Steps:
 # access s3 to build DF 1
 # Input DF 1 into function that builds DF 2 & DF 3
 # Input DF 1 into function that builds DF 4
 # Input DF 3 & DF 4 into function to return DF 5


    

Weekday CBL

1. connect to s3 bucket
2. Calculate CBL basis
    a) event period load over last 30 days = cbl window
    b) iterate backwards to find the 10 eligible weekdays for basis calculation
    c) Take 5 days out of 10 with highest average usage for event period
    d) Average usage for each hour from those 5 days

Weekend CBL

1. connect to s3 bucket
2. 3 most recent weekend days = cbl window
3. 2 highest days, average for each hour of event period = cbl basis


Adjustment Factor

1. adjustment period is 2 hour preiod beginning 4 hours prior to start of event
2. Apply average day CBL process to adjusment window
3. divide by average CBL = adjustment factor